# NLP Practical Exam — Text Processing + Language Modeling (90 minutes)

**Instructions**
- Work in this notebook only.
- Write short, clear comments to justify *tool choices* (regex vs NLTK, etc.).
- Do **not** use external NLP libraries beyond **NLTK**, **NumPy**, **PyTorch** (PyTorch not needed here).
- Keep outputs readable (print key variables).

**Total: 10 points**


## Given text

```python
text = ("In mid-February 2026, the CEO of OpenAI, Sam Altman, visited Barcelona. He is 1.86m tall and met with researchers from U.P.C. and U.N.E.S.C.O. A report valued the project at $3.2 billion.")
```

> Treat the text as *synthetic exam data* (no fact-checking needed).


## Questions

1. **(1 pt)** Sentence splitting using **regex + NLTK**.
2. **(1 pt)** Regex normalization: acronyms, height meters→centimeters, money `$X.Y billion` → `x point y billion` (words).
3. **(1 pt)** Lowercase **except** proper nouns; join multiword proper nouns with underscore (e.g., `Sam Altman → Sam_Altman`). Keep acronyms uppercase.
4. **(1 pt)** Tokenize (tool of your choice).
5. **(1 pt)** Remove stopwords (tool of your choice); keep entity tokens.
6. **(1 pt)** Create bigrams with pure Python.
7. **(2 pt)** Build a bigram LM (MLE) and `predict_next(prev_word, top_k=3)`.

8. **(2 pt)** Implement a simple **BPE** on: `corpus = "low lower newest widest"` (≥5 merges or until no merges).
9. **(1 pt)** Compute Accuracy/Precision/Recall/F1 for an invented confusion matrix (explain with comments).


In [1]:
import re
import math
import nltk
from collections import Counter, defaultdict

# NLTK downloads (safe to run multiple times)
nltk.download("punkt", quiet=True)
nltk.download("stopwords", quiet=True)

from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords

text = ("In mid-February 2026, the CEO of OpenAI, Sam Altman, visited Barcelona. "
        "He is 1.86m tall and met with researchers from U.P.C. and U.N.E.S.C.O. "
        "A report valued the project at $3.2 billion.")

print(text)


In mid-February 2026, the CEO of OpenAI, Sam Altman, visited Barcelona. He is 1.86m tall and met with researchers from U.P.C. and U.N.E.S.C.O. A report valued the project at $3.2 billion.


## Q1

In [3]:
# Q1 (1 pt): Sentence splitting (regex + NLTK)
# - Use regex to protect acronyms like U.P.C. so they don't break sentence boundaries.
# - Then use nltk.sent_tokenize.
#
# Return: sentences (list of strings)

# TODO: implement protect_acronym_dots and restore_acronym_dots (or equivalent)
# TODO: apply sent_tokenize
sentences = None

# print(sentences)

import re
from nltk.tokenize import sent_tokenize

def protect_acronym_dots(raw_text):
    # This regex looks for a dot preceded by an uppercase letter 
    # and followed by another uppercase letter. 
    # It replaces that dot with a placeholder like "<DOT>".
    return re.sub(r'(?<=[A-Z])\.(?=[A-Z])', '<DOT>', raw_text)

def restore_acronym_dots(sentence_list):
    # This goes through the list of sentences and changes the placeholder back to a dot
    return [sentence.replace('<DOT>', '.') for sentence in sentence_list]

# Hide the internal acronym dots
protected_text = protect_acronym_dots(text)

# Split into sentences using NLTK
raw_sentences = sent_tokenize(protected_text)

# Bring the dots back
sentences = restore_acronym_dots(raw_sentences)

# Check the result
for i, s in enumerate(sentences):
    print(f"Sentence {i+1}: {s}")


Sentence 1: In mid-February 2026, the CEO of OpenAI, Sam Altman, visited Barcelona.
Sentence 2: He is 1.86m tall and met with researchers from U.P.C.
Sentence 3: and U.N.E.S.C.O.
Sentence 4: A report valued the project at $3.2 billion.


## Q2

In [4]:
# Q2 (1 pt): Regex normalization
# Convert:
#  - U.P.C. -> UPC, U.N.E.S.C.O. -> UNESCO (general rule: remove dots in acronyms)
#  - 1.86m -> 186 centimeters (general: X.YZm -> int(round(float(X.YZ)*100)) centimeters)
#  - $3.2 billion -> three point two billion  (digits 0-9 are enough)
#
# Return: text_norm

text_norm = None

# print(text_norm)

import re

text_norm = text

# 1. Acronyms: U.P.C. -> UPC
# The regex looks for one or more uppercase letters followed by a dot.
# replace the dots with nothing in the matched string.
text_norm = re.sub(r'([A-Z]\.)+', lambda match: match.group(0).replace('.', ''), text_norm)

# 2. Height: 1.86m -> 186 centimeters
# Capture the decimal number in group 1, convert it to float, 
# multiply by 100, round it, and turn it into an integer.
text_norm = re.sub(r'(\d+\.\d+)m\b', 
                   lambda match: f"{int(round(float(match.group(1)) * 100))} centimeters", 
                   text_norm)

# 3. Money: $3.2 billion -> three point two billion
# Create a simple dictionary to map the digits.
num_to_word = {
    '0': 'zero', '1': 'one', '2': 'two', '3': 'three', '4': 'four',
    '5': 'five', '6': 'six', '7': 'seven', '8': 'eight', '9': 'nine'
}

# Capture the two digits separately and replace them using our dictionary.
def replace_money(match):
    digit1 = num_to_word[match.group(1)]
    digit2 = num_to_word[match.group(2)]
    return f"{digit1} point {digit2} billion"

text_norm = re.sub(r'\$(\d)\.(\d)\s+billion', replace_money, text_norm)

print(text_norm)

In mid-February 2026, the CEO of OpenAI, Sam Altman, visited Barcelona. He is 186 centimeters tall and met with researchers from UPC and UNESCO A report valued the project at three point two billion.


## Q3

In [5]:
# Q3 (1 pt): Lowercase except proper nouns + underscore multiword proper nouns
# Requirements:
# - Convert to lowercase except:
#   - Acronyms (ALL CAPS) stay uppercase (e.g., UNESCO, UPC, CEO)
#   - MixedCase tokens stay as-is (e.g., OpenAI)
#   - Multiword proper nouns joined with underscore (Sam Altman -> Sam_Altman) and preserved
#
# Return: text_case

text_case = None

# print(text_case)

import re

# Assuming text_norm is coming from the previous Q2 solution
# text_norm = "In mid-February 2026, the CEO of OpenAI, Sam Altman, visited Barcelona. He is 186 centimeters tall and met with researchers from UPC and UNESCO. A report valued the project at three point two billion."

text_case = text_norm

# 1. Multiword proper nouns (e.g., "Sam Altman" -> "Sam_Altman")
# Look for two or more consecutive capitalized words and replace the spaces with underscores.
text_case = re.sub(r'\b([A-Z][a-z]+(?:\s+[A-Z][a-z]+)+)\b', 
                   lambda m: m.group(1).replace(' ', '_'), 
                   text_case)

# 2. Lowercase everything else except the specific rules
def smart_lowercase(match):
    word = match.group(0)
    
    # Rule A: Acronyms (ALL CAPS). 
    # (Use len > 1 so we don't accidentally keep a starting 'A' uppercase)
    if word.isupper() and len(word) > 1:
        return word
        
    # Rule B: MixedCase (e.g., OpenAI). 
    # Check if there's any uppercase letter after the very first character.
    if any(char.isupper() for char in word[1:]):
        return word
        
    # Rule C: Multiword proper nouns.
    # Since already added the underscore in step 1, I just check for it.
    if '_' in word:
        return word
        
    # Rule D: Single proper nouns (like "Barcelona" or "February").
    # If it is capitalized, need to check if it's at the start of a sentence.
    if word.istitle():
        # Get the text right before this word to see where I am
        text_before = text_case[:match.start()].strip()
        
        # If there's no text before, or the text before ends with a dot, 
        # it means I am at the beginning of a sentence. So, we lowercase it.
        if not text_before or text_before.endswith('.'):
            return word.lower() # "In", "He", "A" become "in", "he", "a"
        else:
            return word
            
    # For absolutely everything else, just return the lowercase version
    return word.lower()

# Apply to every word (letters and underscores)
text_case = re.sub(r'[a-zA-Z_]+', smart_lowercase, text_case)

print(text_case)

in mid-February 2026, the CEO of OpenAI, Sam_Altman, visited Barcelona. he is 186 centimeters tall and met with researchers from UPC and UNESCO A report valued the project at three point two billion.


## Q4

In [6]:
# Q4 (1 pt): Tokenization
# Use a tokenizer of your choice (e.g., nltk.word_tokenize).
# Return: tokens (list)

tokens = None

# print(tokens)

from nltk.tokenize import word_tokenize

# Assuming text_case contains the string from the end of Q3
# text_case = "in mid-February 2026, the CEO of OpenAI, Sam_Altman, visited Barcelona. he is 186 centimeters tall and met with researchers from UPC and UNESCO. a report valued the project at three point two billion."

# Use NLTK's word_tokenize to split the string into a list of tokens
tokens = word_tokenize(text_case)

# Print the tokens
print(tokens)

['in', 'mid-February', '2026', ',', 'the', 'CEO', 'of', 'OpenAI', ',', 'Sam_Altman', ',', 'visited', 'Barcelona', '.', 'he', 'is', '186', 'centimeters', 'tall', 'and', 'met', 'with', 'researchers', 'from', 'UPC', 'and', 'UNESCO', 'A', 'report', 'valued', 'the', 'project', 'at', 'three', 'point', 'two', 'billion', '.']


## Q5

In [ ]:
# Q5 (1 pt): Stopword removal
# - Remove English stopwords
# - Do NOT remove entity tokens like OpenAI, Sam_Altman, Barcelona, UNESCO, UPC
# Return: tokens_nostop

tokens_nostop = None

# print(tokens_nostop)


## Q6

In [ ]:
# Q6 (1 pt): Bigrams with pure Python (no NLTK bigrams helper)
# Return: bigrams = [(w1, w2), ...]

bigrams = None

# print(bigrams)


## Q7

In [ ]:
# Q7 (2 pt): Bigram Language Model + next-word prediction
# Build:
# - bigram_counts[(w1,w2)]
# - context_counts[w1]
# - model[w1][w2] = P(w2|w1) = count(w1,w2)/count(w1)
#
# Then implement:
# def predict_next(prev_word, model, top_k=3): -> list[(next_word, prob)] sorted

bigram_counts = None
context_counts = None
model = None

def predict_next(prev_word, model, top_k=3):
    # TODO
    return None

# Example:
# print(predict_next("OpenAI", model, top_k=3))


## Q8

In [ ]:
# Q8 (2 pt): Simple BPE (Byte Pair Encoding) on a tiny corpus
corpus = "low lower newest widest"

# Requirements:
# - Represent each word as characters + </w>
# - Compute pair frequencies (weighted by word frequency)
# - Merge most frequent pair
# - Do at least 5 merges (or stop if no pairs)
#
# Deliver:
# - merges: list of merges in order
# - final segmented version of each word

merges = None

# TODO: implement BPE helper functions:
# - get_vocab_from_corpus
# - get_pair_frequencies
# - merge_pair_in_vocab

# print(merges)


## Q9

In [ ]:
# Q9 (1 pt): Metrics — Accuracy, Precision, Recall, F1
# Invent a confusion matrix (TP, FP, FN, TN) and compute metrics.
# Explain each formula briefly in comments.

TP = None
FP = None
FN = None
TN = None

accuracy = None
precision = None
recall = None
f1 = None

# print(accuracy, precision, recall, f1)
